# 3-Manifolds and Log VOA
## Example section companion notebook

Attaching a Wilson operator to the central node of Seifert manifolds leads to different $\hat{Z}$ invariants.  In this example notebook, we replicate the examples of Wilson lines attached to the central of a Seifert manifold plumbing graph in the spherical and pseudospherical cases.

In [1]:
load("../../pySeifert.sage")

We start with the spherical case:

In [2]:
S = Seifert([-1,3,5,2,7,1,9])
B = S.boundary_conditions(["A",2],basis = "weight"); B

[[(-1, -1), (0, 0), (1, 1), (0, 0), (1, 1), (1, 1)]]

To compute the modified $\hat{Z}$ invariant we must list all $\tilde{\chi}$:

In [3]:
WG = [g.transpose() for g in weyl_group(["A",2])]
WL = [det(g) for g in WG]
wh_list = list(itertools.product(*[WG if d == 1 else [matrix(2)] for d in S.deg]))
whl_list = list(itertools.product(*[WL if d == 1 else [1] for d in S.deg])) 
Chit_list = [product(l)*S.chi_tilde(["A",2],wh,B[0],300,basis = "weight") for wh,l in zip(wh_list,whl_list)]

In this notebook we attach a Wilson operator in the $\vec\nu=(2,2)$ representation to the central node.

To compute the $\hat{Z}$, we must sum the $\tilde{\chi}$ multiplied by 
$$
    m_{\vec\sigma}^{(\vec\nu)}\sum_{w\in W}e^{\left<\vec\xi,w(\vec\sigma)\right>}
$$
where $m_{\vec\sigma}^{(\vec{\nu})}$ is the multiplicity of $\vec\sigma$ in the representation of highest weight $\vec{\nu}$, for each $\vec{\sigma}$ in the set of dominant integral weights.

In [8]:
nu = vector([2,2])
wilson_center_pref = 0
for sig in itertools.product(range(4),repeat = 2):
    sig = vector(sig)
    if (sig[0]-sig[1])%3 != 0:
        continue
    m_sig = freudenthal_multiplicity(["A",2],nu, sig)
    for w in WG:
        w_sig = w*sig
        wilson_center_pref += m_sig*z0^w_sig[0]*z1^w_sig[1]
Zhat_integrand = wilson_center_pref * sum(Chit_list)

Take out the constant term after multiplying by a sufficiently expandend expansion of the denominator

In [15]:
Zhat = const_term(Zhat_integrand); Zhat

216*q^296 - 216*q^294 + 216*q^287 + 216*q^284 - 108*q^283 - 432*q^280 + 216*q^278 + 432*q^272 - 216*q^267 + 216*q^266 - 216*q^262 + 216*q^260 - 216*q^259 + 216*q^254 + 216*q^252 + 432*q^248 - 216*q^238 + 216*q^236 - 216*q^235 - 216*q^232 + 216*q^228 - 216*q^226 - 108*q^219 - 432*q^211 - 216*q^210 - 216*q^202 - 216*q^196 - 216*q^195 + 216*q^191 - 216*q^190 + 216*q^188 - 108*q^187 + 216*q^179 + 432*q^164 + 216*q^159 + 216*q^158 + 216*q^144 - 216*q^142 - 216*q^126 - 216*q^118 + 108*q^95

We compute the same with triplet characters. We start from computing $\vec A_{\hat w}$ and $\vec \mu$.

In [16]:
Aw_list = list()
mu_list = list()
rho = weyl_vector(["A",2])
for wh in wh_list:
    Aw = -1/S.Minv[0,0]*S.Minv[0]*matrix([w*rho for w in wh])
    mu = 1/sqrt(S.m)*(rho+S.m*Aw)
    Aw_list.append(Aw)
    mu_list.append(mu)

and, with these the triplet characters

In [17]:
tri_ch_list = list()
for mu,l in zip(mu_list,whl_list):
        tri_ch_list.append(product(l)*triplet_character(["A",2],vector([0,0]),mu,S.m,S.f,300,basis = "weight", qvar = None))

... and then assemble the $\hat {Z}$ and integrate.

In [27]:
Zhat_ch_integrand = wilson_center_pref*sum(tri_ch_list)
Zhat_ch = const_term(Zhat_ch_integrand)*q^(S.delta(["A",2]))
Zhat_ch

108*(2*q^(94159/315) + 2*q^(92899/315) - 2*q^(92269/315) + 2*q^(90064/315) + 2*q^(89119/315) - q^(88804/315) - 4*q^(87859/315) + 2*q^(87229/315) + 4*q^(85339/315) - 2*q^(83764/315) + 2*q^(83449/315) - 2*q^(82189/315) + 2*q^(81559/315) - 2*q^(81244/315) + 2*q^(79669/315) + 2*q^(79039/315) + 4*q^(77779/315) - 2*q^(74629/315) + 2*q^(73999/315) - 2*q^(73684/315) - 2*q^(72739/315) + 2*q^(71479/315) - 2*q^(70849/315) - q^(68644/315) - 4*q^(66124/315) - 2*q^(65809/315) - 2*q^(63289/315) - 2*q^(61399/315) - 2*q^(61084/315) + 2*q^(59824/315) - 2*q^(59509/315) + 2*q^(58879/315) - q^(58564/315) + 2*q^(56044/315) + 4*q^(51319/315) + 2*q^(49744/315) + 2*q^(49429/315) + 2*q^(45019/315) - 2*q^(44389/315) - 2*q^(39349/315) - 2*q^(36829/315) + q^(29584/315))*q^(341/315)

Finally, we assess that the expressions for the $\hat{Z}$ are the same:

In [28]:
expand(Zhat_ch-Zhat).series(q,300)

Order(q^300)

The pseudospherical case is similar. The $\hat{Z}$ integrand and the triplet characters can be rapidly computed:

In [31]:
S = Seifert([-1,1,2,1,3,1,9])
B = S.boundary_conditions(["A",2],basis="weight")
b0 = B[0]
WG = [g.transpose() for g in weyl_group(["A",2])]
WL = [det(g) for g in WG]
wh_list = list(itertools.product(*[WG if d == 1 else [matrix(2)] for d in S.deg]))
whl_list = list(itertools.product(*[WL if d == 1 else [1] for d in S.deg]))
Chit_list = [product(l)*S.chi_tilde(["A",2],wh,b0,100,basis = "weight") for wh,l in zip(wh_list,whl_list)]
Zhat_integrand = wilson_center_pref * sum(Chit_list)
Zhat = const_term(Zhat_integrand)

As we did above, we compute the indices of the triplet characters. We only compute triplet characters for $\hat{w}$ corresponding to $\tilde{\chi}$ that do not vanish due to $S_{\hat{w},\vec{\underline{b}}}$

In [34]:
Aw_list = list()
mu_list = list()
rho = weyl_vector(["A",2])
for wh in wh_list:
    Aw = -1/S.Minv[0,0]*S.Minv[0]*matrix([w*rho for w in wh])
    mu = 1/sqrt(S.m)*(rho+S.m*Aw)
    Aw_list.append(Aw)
    mu_list.append(mu)

tri_ch_list = list()
for mu,l,chit in zip(mu_list,whl_list,Chit_list):
    if chit != 0:
        tri_ch_list.append(product(l)*triplet_character(["A",2],vector([0,0]),mu,S.m,S.f,100,basis = "weight", qvar = None))
ZhatCh_integrand = wilson_center_pref*sum(tri_ch_list)*q^(S.delta(["A",2]))
ZhatCh = const_term(ZhatCh_integrand)


In [38]:
(Zhat-ZhatCh).series(q,100)

Order(q^100)